In [ ]:
import pandas as pd
import json

In [ ]:
places_df = pd.read_json("../../data/places/nyt-sum-places.json")

In [ ]:
places_df.head(5)

In [ ]:
places_df.loc[:,['name', 'count']]

In [ ]:
places_categories_df = places_df.copy()

In [ ]:
data = []
for index, location in places_categories_df.iterrows():
    dict = location["related_keywords"]
    # Ugly but works 🤪
    if location["name"] in dict: del dict[location["name"]]
    if location["name"].lower() in dict: del dict[location["name"].lower()]
    if location["name"].upper() in dict: del dict[location["name"].upper()]
    if location["name"].capitalize() in dict: del dict[location["name"].capitalize()]
    print
    keys = dict.keys()
    values = dict.values()
    for key in keys:

        data.append((location["name"], key, dict[key]))


In [ ]:
df_expanded = pd.DataFrame(data, columns=('location','keyword', 'value'))

In [ ]:
unique_keywords = list(df_expanded["keyword"].unique())

with open('unique_keywords.txt', 'w') as f:
    for keyword in unique_keywords:
        f.write(f"{keyword}\n")

In [ ]:
grouped = df_expanded.groupby(by="location")

In [ ]:
grouped = df_expanded.groupby("location")

result = []
for location, group in grouped:
    print(f"Location: {location}")
    original_data_instance = places_categories_df[places_categories_df["name"] == location].to_dict(orient='records')[0]
    original_data_instance.pop("related_keywords", None)  # Exclude "related_keywords"
    
    # Wrap "timeline" in a list
    if "timeline" in original_data_instance:
        original_data_instance["timeline"] = [original_data_instance["timeline"]]
    
    top_keywords = group.nlargest(20, 'value')[["keyword", "value"]]
    print(top_keywords)
    print("\n")
    result.append({
        "location": location,
        "count": original_data_instance.get("count", 0),
        "top_keywords": top_keywords.to_dict(orient='records')
    })

# Export to JSON
with open('../../data/top_keywords_by_location.json', 'w') as f:
    json.dump(result, f)